In [1]:
#import string
import re # Regex (expressão regular)
!pip install unidecode
from unicodedata import normalize # utíl para tratamento de texto e compatibilidade
!pip install pandas
import pandas as pd
!pip install nltk
import nltk # Ferramentas de PLN
!pip install sklearn
import sklearn # Ferramentas de aprendizado de máquina
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
#from sklearn.metrics import roc_auc_score
!pip install seaborn
import seaborn as sns
!pip install matplotlib
import matplotlib.pyplot as plt # Plotagem de gráficosX_train_count

print("\n" + '\033[1m' + '\033[93m' + "Importações feitas com sucesso!")


Importações feitas com sucesso!


## Dados
* As seguintes CSV foram baixados em [Portuguese Tweets for Sentiment Analysis](https://www.kaggle.com/augustop/portuguese-tweets-for-sentiment-analysis) criado por [augustop](https://www.kaggle.com/augustop) 

In [2]:
# https://www.kaggle.com/augustop/portuguese-tweets-for-sentiment-analysis

arquivo = 'Train50.csv'

df_tweets = pd.read_csv(arquivo, sep=";", usecols=['tweet_text','sentiment'], index_col=None, header=0, dtype={"sentiment":"int16"})

df_tweets

,tweet_text,sentiment
0,"@queren_renata Bom dia, ótima semana :)",1
1,o frio vai voltar finalmente :)),1
2,@kizqe damn fiquei a sorrir mto :),1
3,"@lopes85 @_Goalpoint ""gentes"" do benfiquistão ...",1
4,@crlhemely Exatamente! E a outra metade é modi...,1
...,...,...
39995,Vao cancelar minhas aulas de manhã :(,0
39996,Triste a rejeição do Haddad estar próxima a do...,0
39997,@vicevelyn_ Infelizmente tbm n :(,0
39998,@r9carloss No entendiii ptm :(,0


In [3]:
# é importante que os dados estejam proximos para não criar um 'Bias' muito grande a um tipo
df_tweets['sentiment'].value_counts()
# 1 = Positivo 🙂
# 0 = Negativo 🙁

1    20000
0    20000
Name: sentiment, dtype: int64

In [4]:
df_tweets.sentiment

0        1
1        1
2        1
3        1
4        1
        ..
39995    0
39996    0
39997    0
39998    0
39999    0
Name: sentiment, Length: 40000, dtype: int16

//# como o computador só entende números nós precisamos tranformar o maximo possivel em números
df_tweets['score'] = df_tweets['sentiment'].replace(['Negativo', 'Positivo'], [0, 1])

df_tweets

In [5]:
# Deixando tudo em minúsculo
df_tweets['tweet_text'] = df_tweets['tweet_text'].str.lower()

df_tweets['tweet_text']

0                  @queren_renata bom dia, ótima semana :)
1                         o frio vai voltar finalmente :))
2                       @kizqe damn fiquei a sorrir mto :)
3        @lopes85 @_goalpoint "gentes" do benfiquistão ...
4        @crlhemely exatamente! e a outra metade é modi...
                               ...                        
39995                vao cancelar minhas aulas de manhã :(
39996    triste a rejeição do haddad estar próxima a do...
39997                    @vicevelyn_ infelizmente tbm n :(
39998                       @r9carloss no entendiii ptm :(
39999                     só queria estar na minha cama :(
Name: tweet_text, Length: 40000, dtype: object

In [6]:
# Removendo tudo entre []
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='\[.*?\]', value='', regex=True)

df_tweets['tweet_text']

0                  @queren_renata bom dia, ótima semana :)
1                         o frio vai voltar finalmente :))
2                       @kizqe damn fiquei a sorrir mto :)
3        @lopes85 @_goalpoint "gentes" do benfiquistão ...
4        @crlhemely exatamente! e a outra metade é modi...
                               ...                        
39995                vao cancelar minhas aulas de manhã :(
39996    triste a rejeição do haddad estar próxima a do...
39997                    @vicevelyn_ infelizmente tbm n :(
39998                       @r9carloss no entendiii ptm :(
39999                     só queria estar na minha cama :(
Name: tweet_text, Length: 40000, dtype: object

In [7]:
# remover links
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='https?://[A-Za-z0-9./]+', value='', regex=True)
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='http?://[A-Za-z0-9./]+', value='', regex=True)
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='wwww?://[A-Za-z0-9./]+', value='', regex=True)
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='ftp?://[A-Za-z0-9./]+', value='', regex=True)

In [8]:
# Removendo números
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='\d+', value='', regex=True)

df_tweets['tweet_text']

0                  @queren_renata bom dia, ótima semana :)
1                         o frio vai voltar finalmente :))
2                       @kizqe damn fiquei a sorrir mto :)
3        @lopes @_goalpoint "gentes" do benfiquistão nã...
4        @crlhemely exatamente! e a outra metade é modi...
                               ...                        
39995                vao cancelar minhas aulas de manhã :(
39996    triste a rejeição do haddad estar próxima a do...
39997                    @vicevelyn_ infelizmente tbm n :(
39998                        @rcarloss no entendiii ptm :(
39999                     só queria estar na minha cama :(
Name: tweet_text, Length: 40000, dtype: object

In [9]:
# Removendo #rashtags
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace=r'(\#\w+)', value='', regex=True)

df_tweets['tweet_text']

0                  @queren_renata bom dia, ótima semana :)
1                         o frio vai voltar finalmente :))
2                       @kizqe damn fiquei a sorrir mto :)
3        @lopes @_goalpoint "gentes" do benfiquistão nã...
4        @crlhemely exatamente! e a outra metade é modi...
                               ...                        
39995                vao cancelar minhas aulas de manhã :(
39996    triste a rejeição do haddad estar próxima a do...
39997                    @vicevelyn_ infelizmente tbm n :(
39998                        @rcarloss no entendiii ptm :(
39999                     só queria estar na minha cama :(
Name: tweet_text, Length: 40000, dtype: object

In [10]:
# Removendo @Mentions
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace=r'(\@\w+)', value='', regex=True)

df_tweets['tweet_text']

0                                 bom dia, ótima semana :)
1                         o frio vai voltar finalmente :))
2                              damn fiquei a sorrir mto :)
3          "gentes" do benfiquistão não gostam de futeb...
4               exatamente! e a outra metade é modinha :))
                               ...                        
39995                vao cancelar minhas aulas de manhã :(
39996    triste a rejeição do haddad estar próxima a do...
39997                                infelizmente tbm n :(
39998                                  no entendiii ptm :(
39999                     só queria estar na minha cama :(
Name: tweet_text, Length: 40000, dtype: object

In [11]:
# Removendo a pontuação (EX: "!?.,/|#$%¨&")
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace=r'[^\w\s]', value='', regex=True)

df_tweets['tweet_text']

0                                    bom dia ótima semana 
1                            o frio vai voltar finalmente 
2                                damn fiquei a sorrir mto 
3          gentes do benfiquistão não gostam de futebol...
4                   exatamente e a outra metade é modinha 
                               ...                        
39995                  vao cancelar minhas aulas de manhã 
39996    triste a rejeição do haddad estar próxima a do...
39997                                  infelizmente tbm n 
39998                                    no entendiii ptm 
39999                       só queria estar na minha cama 
Name: tweet_text, Length: 40000, dtype: object

In [12]:
# Removendo acentuação
df_tweets['tweet_text'] = df_tweets['tweet_text'].str.normalize('NFKD').str.encode('ASCII', errors='ignore').str.decode('UTF-8')

df_tweets['tweet_text']

0                                    bom dia otima semana 
1                            o frio vai voltar finalmente 
2                                damn fiquei a sorrir mto 
3          gentes do benfiquistao nao gostam de futebol...
4                   exatamente e a outra metade e modinha 
                               ...                        
39995                  vao cancelar minhas aulas de manha 
39996    triste a rejeicao do haddad estar proxima a do...
39997                                  infelizmente tbm n 
39998                                    no entendiii ptm 
39999                       so queria estar na minha cama 
Name: tweet_text, Length: 40000, dtype: object

In [13]:
# Uma stopword pode ser considerada uma palavra irrelevante para a análise
nltk.download('stopwords')
# RSLP(Removedor de Sufixos da Língua Portuguesa)
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\natan\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [14]:
# Separa por tokens (tokens = palavras)
#tokenizer = nltk.tokenize.WordPunctTokenizer()

In [15]:
# O processo de stemming consiste em reduzir a palavra à sua raiz (sem levar em conta a classe gramatical)
##stemmer = nltk.stem.RSLPStemmer() # RSLP(Removedor de Sufixos da Língua Portuguesa)
##df_tweets['tweet_text_stemmed'] = df_tweets['tweet_text'].apply(lambda x: [stemmer.stem(y) for y in x])

///# Nesse passo é criado uma função que faz a Tokenização e o Stemming
def criarBagOfWords(text, stop_words=stopWords):
    tokens = tokenizer.tokenize(text)
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

df_tweets['bag_of_words'] = df_tweets['tweet_text'].apply(criarBagOfWords)

df_tweets['bag_of_words']

In [16]:
# Criando uma lista de stopWords
stopWords = set(nltk.corpus.stopwords.words('portuguese'))

vetorizador = CountVectorizer(stop_words=stopWords)
#
X = vetorizador.fit_transform(X)

NameError: name 'X' is not defined

In [20]:
X = vetorizador.fit_transform(df_tweets.tweet_text)
y = df_tweets.sentiment

# Separa em amostras de treino -> X_train, y_train e amostras de teste -> X_test y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [22]:
#X_train_count = vetorizador.fit_transform(X_train.values)
#X_train_count = X_train_count.toarray()[:3]

AttributeError: lower not found

In [23]:
modelo = naive_bayes.MultinomialNB()
modelo.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
frases = [
    'um bom dia',
    'que coisa horrivel'
]

frases_count = vetorizador.transform(frases)

In [28]:
modelo.predict(frases_count)

array([1, 0], dtype=int16)

In [29]:
modelo.score(X_test,y_test)

0.7496